In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

# Data preparation

Descargamos dataset ReDial:

In [ ]:
!wget https://github.com/ReDialData/website/raw/data/redial_dataset.zip

--2025-06-30 20:45:12--  https://github.com/ReDialData/website/raw/data/redial_dataset.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ReDialData/website/data/redial_dataset.zip [following]
--2025-06-30 20:45:13--  https://raw.githubusercontent.com/ReDialData/website/data/redial_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5765261 (5.5M) [application/zip]
Saving to: ‘redial_dataset.zip’

redial_dataset.zip  100%[===================>]   5.50M  --.-KB/s    in 0.03s   

2025-06-30 20:45:13 (161 MB/s) - ‘redial_dataset.zip’ saved [5765261/5765261]



In [ ]:
!unzip /content/redial_dataset.zip

Archive:  /content/redial_dataset.zip
  inflating: movies_with_mentions.csv  
  inflating: test_data.jsonl         
  inflating: train_data.jsonl        


Cargamos el dataset a python con código obtenido en [este link](https://github.com/ReDialData/website/blob/data/load_data.py).

In [ ]:
import zipfile
import json

with zipfile.ZipFile('redial_dataset.zip', 'r') as z:
    z.extractall()

def read_jsonl(path):
    with open(path, "r", encoding="utf-8") as fh:
        return [json.loads(line) for line in fh]

train_raw = read_jsonl("train_data.jsonl")
test_raw = read_jsonl("test_data.jsonl")

print(f"Loaded {len(train_raw)} train conversations")
print(f"Loaded {len(test_raw)}  test conversations")

Loaded 10006 train conversations
Loaded 1342  test conversations


In [ ]:
import re

def decode_movie_mentions(text, movies):
    """Replace @12345 with its movie title if available."""
    movie_pattern = re.compile(r"@(\d+)")
    return movie_pattern.sub(lambda m: movies.get(m.group(1), m.group(0)), text)

def redial_to_messages(example):
    """Return a dict with a *single* key 'messages'."""
    movies = example["movieMentions"]
    user_id, assistant_id = example["initiatorWorkerId"], example["respondentWorkerId"]

    chat = []
    for m in example["messages"]:
        role = "user" if m["senderWorkerId"] == user_id else "assistant"
        chat.append({
            "role": role,
            "content": decode_movie_mentions(m["text"], movies)
        })

    if chat[-1]["role"] == "assistant":
        chat.append({"role": "user",
                     "content": "Any other recommendation?"})
    return {"messages": chat}

processed_train = [redial_to_messages(e) for e in train_raw]
processed_test  = [redial_to_messages(e) for e in test_raw]

In [ ]:
processed_train[0]

{'messages': [{'role': 'user',
   'content': "Hi there, how are you? I'm looking for movie recommendations"},
  {'role': 'assistant',
   'content': 'I am doing okay. What kind of movies do you like?'},
  {'role': 'user',
   'content': 'I like animations like The Triplets of Belleville (2003) and Waking Life (2001)'},
  {'role': 'user', 'content': 'I also enjoy Mary and Max (2009)'},
  {'role': 'user', 'content': 'Anything artistic'},
  {'role': 'assistant',
   'content': 'You might like The Boss Baby (2017) that was a good movie.'},
  {'role': 'user', 'content': "What's it about?"},
  {'role': 'assistant',
   'content': 'It has Alec Baldwin it is about a baby that works for a company and gets adopted it is very funny'},
  {'role': 'user', 'content': 'That seems like a nice comedy'},
  {'role': 'user',
   'content': 'Do you have any animated recommendations that are a bit more dramatic? Like A Scanner Darkly  (2006) for example'},
  {'role': 'user',
   'content': 'I like comedies but I 

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_list(processed_train)
test_dataset  = Dataset.from_list(processed_test)

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

max_seq_length = 2048

def initialize_model_and_tokenizer(base_model_name: str = "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", max_seq_length: int = max_seq_length, dtype = None, load_in_4bit: bool = True,):

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name     = base_model_name,
        max_seq_length = max_seq_length,
        dtype          = dtype,
        load_in_4bit   = load_in_4bit,
    )


    model = FastLanguageModel.get_peft_model(
        model,
        r               = 16,
        target_modules  = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha      = 16,
        lora_dropout    = 0.0,
        bias            = "none",
        use_gradient_checkpointing = "unsloth",
        random_state    = 3407,
        use_rslora      = False,
        loftq_config    = None,
    )

    tokenizer = get_chat_template(tokenizer, chat_template = "qwen-2.5")

    return model, tokenizer

model, tokenizer = initialize_model_and_tokenizer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.9: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.6.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
def formatting_prompts_func(examples):
  convos = examples["messages"]
  texts = [
      tokenizer.apply_chat_template(
          convo,
          tokenize              = False,
          add_generation_prompt = True
      )
      for convo in convos
  ]
  return {"text": texts}

train_dataset = train_dataset.map(
  formatting_prompts_func,
  batched=True,
  remove_columns=[c for c in train_dataset.column_names if c != "messages"],
)

test_dataset = test_dataset.map(
  formatting_prompts_func,
  batched=True,
  remove_columns=[c for c in test_dataset.column_names if c != "messages"],
)

Map:   0%|          | 0/10006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1342 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0]

{'messages': [{'content': "Hi there, how are you? I'm looking for movie recommendations",
   'role': 'user'},
  {'content': 'I am doing okay. What kind of movies do you like?',
   'role': 'assistant'},
  {'content': 'I like animations like The Triplets of Belleville (2003) and Waking Life (2001)',
   'role': 'user'},
  {'content': 'I also enjoy Mary and Max (2009)', 'role': 'user'},
  {'content': 'Anything artistic', 'role': 'user'},
  {'content': 'You might like The Boss Baby (2017) that was a good movie.',
   'role': 'assistant'},
  {'content': "What's it about?", 'role': 'user'},
  {'content': 'It has Alec Baldwin it is about a baby that works for a company and gets adopted it is very funny',
   'role': 'assistant'},
  {'content': 'That seems like a nice comedy', 'role': 'user'},
  {'content': 'Do you have any animated recommendations that are a bit more dramatic? Like A Scanner Darkly  (2006) for example',
   'role': 'user'},
  {'content': 'I like comedies but I prefer films with a

# Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/10006 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
import torch

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.01 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,006 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 17,432,576 of 1,700,000,000 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,6.779100
2,7.062900
3,7.015200
4,5.981700
5,4.720900
6,4.149900
7,3.415600
8,3.026000
9,2.908800
10,2.472400


In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

513.8698 seconds used for training.
8.56 minutes used for training.
Peak reserved memory = 2.748 GB.
Peak reserved memory for training = 0.738 GB.
Peak reserved memory % of max memory = 18.642 %.
Peak reserved memory for training % of max memory = 5.006 %.


In [ ]:
model.save_pretrained("redial_qwen_model")
tokenizer.save_pretrained("redial_qwen_model")

('redial_qwen_model/tokenizer_config.json',
 'redial_qwen_model/special_tokens_map.json',
 'redial_qwen_model/chat_template.jinja',
 'redial_qwen_model/vocab.json',
 'redial_qwen_model/merges.txt',
 'redial_qwen_model/added_tokens.json',
 'redial_qwen_model/tokenizer.json')

In [ ]:
!zip -r redial_qwen_model.zip redial_qwen_model/

  adding: redial_qwen_model/ (stored 0%)
  adding: redial_qwen_model/tokenizer_config.json (deflated 90%)
  adding: redial_qwen_model/added_tokens.json (deflated 68%)
  adding: redial_qwen_model/merges.txt (deflated 57%)
  adding: redial_qwen_model/adapter_model.safetensors (deflated 8%)
  adding: redial_qwen_model/chat_template.jinja (deflated 71%)
  adding: redial_qwen_model/vocab.json (deflated 61%)
  adding: redial_qwen_model/tokenizer.json (deflated 81%)
  adding: redial_qwen_model/adapter_config.json (deflated 56%)
  adding: redial_qwen_model/special_tokens_map.json (deflated 69%)
  adding: redial_qwen_model/README.md (deflated 66%)


Uncomment if, instead of training the model with the code above, the model is imported as a `.zip`

In [ ]:
!unzip redial_lama_model.zip -d redial_lama_model

Archive:  redial_lama_model.zip
   creating: redial_lama_model/redial_lama_model/
  inflating: redial_lama_model/redial_lama_model/tokenizer_config.json  
  inflating: redial_lama_model/redial_lama_model/adapter_model.safetensors  
  inflating: redial_lama_model/redial_lama_model/chat_template.jinja  
  inflating: redial_lama_model/redial_lama_model/tokenizer.json  
  inflating: redial_lama_model/redial_lama_model/adapter_config.json  
  inflating: redial_lama_model/redial_lama_model/special_tokens_map.json  
  inflating: redial_lama_model/redial_lama_model/README.md  


# Evaluation

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "redial_qwen_model",
    max_seq_length = max_seq_length,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.6.9: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template = "qwen-2.5")

In [ ]:
%%capture
!pip install tqdm
!pip install rouge_score
!pip install evaluate bert_score

In [ ]:
from tqdm import tqdm
import torch
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from collections import Counter
import numpy as np
import nltk
from evaluate import load as load_eval

In [ ]:
test_dataset[0]["messages"]

[{'content': 'Hi I am looking for a movie like Super Troopers (2001)',
  'role': 'user'},
 {'content': 'You should watch Police Academy  (1984)', 'role': 'assistant'},
 {'content': 'Is that a great one? I have never seen it. I have seen American Pie ',
  'role': 'user'},
 {'content': 'I mean American Pie  (1999)', 'role': 'user'},
 {'content': 'Yes Police Academy  (1984) is very funny and so is Police Academy 2: Their First Assignment (1985)',
  'role': 'assistant'},
 {'content': 'It sounds like I need to check them out', 'role': 'user'},
 {'content': 'yes you will enjoy them', 'role': 'assistant'},
 {'content': 'I appreciate your time. I will need to check those out. Are there any others you would recommend?',
  'role': 'user'},
 {'content': 'yes Lethal Weapon (1987)', 'role': 'assistant'},
 {'content': 'Thank you i will watch that too', 'role': 'user'},
 {'content': 'and also Beverly Hills Cop (1984)', 'role': 'assistant'},
 {'content': 'Thanks for the suggestions.', 'role': 'user'},

NOTA: Usamos 456 ejemplos para PEARL. Usaré los mismos para ReDial para tener una comparación justa.

In [ ]:
percentage = 0.2
# Esto hace internamente un muestreo sin tener que barajar todo el dataset
split = test_dataset.train_test_split(test_size=percentage, seed=42)
subset_test_dataset = split["test"]

print(f"Usando {len(subset_test_dataset)} ejemplos para la evaluación.")


Usando 269 ejemplos para la evaluación.


In [ ]:
subset_test_dataset

Dataset({
    features: ['messages', 'text'],
    num_rows: 269
})

In [ ]:
import re

TITLE_RE = re.compile(
    r"(?<=[a-z]\s)([A-Z][A-Za-z0-9\s:,'\-&]+?)\s*\(\d{4}\)",
    re.VERBOSE,
)

def extract_title(text):
    match = TITLE_RE.search(text)
    if match:
        return match.group(1) or match.group(2) or match.group(3)
    return None

import re
import string

def normalize_title(title: str) -> str:
    if not title or not isinstance(title, str):
        return ""

    title = re.sub(r"\s*\(\d{4}\)", "", title)

    title = title.translate(str.maketrans("", "", string.punctuation.replace("'", "")))

    title = title.strip().lower()

    return title


In [ ]:
generation_args = {
    "max_new_tokens": 100,
    "temperature":    0.3,
    "top_p":          0.9,
    "top_k":          50,
    "use_cache":      True,
}

predictions = []
references  = []

for example in tqdm(subset_test_dataset):

    msgs    = example["messages"]
    norm_text = None
    context = example["messages"]

    for message in range(len(msgs)-1, -1, -1):
      if msgs[message]["role"] == "assistant":
        ex = extract_title(msgs[message]["content"])

        if ex:
          norm_text =  msgs[message]
          context = msgs[:message]

          break


    # this returns a Tensor of shape [1, seq_len]
    inputs = tokenizer.apply_chat_template(
        context,
        tokenize              = True,
        add_generation_prompt = True,
        return_tensors        = "pt",
    )
    # inputs is already your input_ids
    input_ids = inputs.cuda()    # no indexing with ["input_ids"]
    attention_mask = None        # if you need one, generate it yourself or ignore

    with torch.no_grad():
        outputs = model.generate(
            input_ids = input_ids,
            **generation_args
        )

    raw = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # drop everything before the assistant tag
    if "<|assistant|>" in raw:
        gen = raw.split("<|assistant|>")[-1].strip()
    else:
        gen = raw.strip()

    predictions.append(gen)


    references.append(norm_text)

100%|██████████| 269/269 [03:49<00:00,  1.17it/s]


In [ ]:
clean_list = []
generated_predicitons = []
for i in range(len(predictions)):
  pred_split = split_dialogue(predictions[i])
  for j in range(len(pred_split)-1, -1, -1):
    if pred_split[j]["role"] == "assistant":
        ex = extract_title(pred_split[j]["content"])
        norm = normalize_title(ex)
        clean_list.append(norm)
        generated_predicitons.append(pred_split[j]["content"])
        break

In [ ]:
references_message = []
for ref in references:
  if ref:
    references_message.append(ref["content"])
  else:
    references_message.append("")

In [ ]:
references[0]

{'content': "You'd love Pandorum (2009) and Event Horizon  (1997)",
 'role': 'assistant'}

In [ ]:
generated_predicitons[0]

'I really like The Matrix (1999)'

In [ ]:
correct = 0
total   = 0
output  = []

empty_string_count = 0

for pred_text, ref_text in zip(clean_list, references_message):


  if ref_text:
    ex = extract_title(ref_text)
    norm = normalize_title(ex)

  pred_title = pred_text
  ref_title  = norm

  if pred_text == "":
    empty_string_count += 1

  hit = pred_title == ref_title
  correct += int(hit)
  total   += 1

  output.append({
      "reference": ref_title,
      "prediction": pred_title,
      "hit": hit
})

recall_at_1 = correct / total if total > 0 else 0
print(f"Recall@1: {recall_at_1:.3f}")
print(f"empty_string_count: {empty_string_count}/{len(clean_list)}")

Recall@1: 0.019
empty_string_count: 160/269


In [ ]:
import json
with open("recall1_eval.json", "w") as f:
    json.dump(output, f, indent=2)


In [ ]:
# 7) Compute BLEU
bleu_score = corpus_bleu([[r] for r in references_message], generated_predicitons)
print(f"BLEU Score: {bleu_score:.4f}")

BLEU Score: 0.1196


In [ ]:
# 8) Compute ROUGE
scorer = rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)
rouge_scores = [scorer.score(r, p) for r, p in zip(references_message, generated_predicitons)]
print("ROUGE-1:", np.mean([s["rouge1"].fmeasure for s in rouge_scores]))
print("ROUGE-2:", np.mean([s["rouge2"].fmeasure for s in rouge_scores]))
print("ROUGE-L:", np.mean([s["rougeL"].fmeasure for s in rouge_scores]))

ROUGE-1: 0.12505760657558124
ROUGE-2: 0.02912726796103556
ROUGE-L: 0.11771083296263414


In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# 9) Compute Distinct-1/2
def distinct(seqs):
    intra1, intra2 = [], []
    uni_all, bi_all = Counter(), Counter()
    for seq in seqs:
        unigrams = Counter(seq)
        bigrams  = Counter(zip(seq, seq[1:]))
        intra1.append(len(unigrams)/ (len(seq)+1e-5))
        intra2.append(len(bigrams)/ (max(len(seq)-1,1)))
        uni_all.update(unigrams)
        bi_all.update(bigrams)
    inter1 = len(uni_all)/sum(uni_all.values())
    inter2 = len(bi_all)/sum(bi_all.values())
    return np.mean(intra1), np.mean(intra2), inter1, inter2

# tokenize references for distinct
import nltk
nltk.download("punkt")
nltk.download('punkt_tab')
tokenized_refs = [nltk.word_tokenize(p) for p in generated_predicitons]
d1_i, d2_i, d1_e, d2_e = distinct(tokenized_refs)
print(f"Distinct-1 intra/inter: {d1_i:.4f}/{d1_e:.4f}")
print(f"Distinct-2 intra/inter: {d2_i:.4f}/{d2_e:.4f}")

Distinct-1 intra/inter: 0.9642/0.1649
Distinct-2 intra/inter: 0.9705/0.4145


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:

bertscore = load_eval("bertscore")
results  = bertscore.compute(
    predictions = generated_predicitons,
    references  = references_message,
    lang        = "en",
    model_type  = "distilbert-base-uncased",
)
print(f"BERTScore F1: {np.mean(results['f1']):.4f}")
print(f"Precision:   {np.mean(results['precision']):.4f}")
print(f"Recall:      {np.mean(results['recall']):.4f}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

BERTScore F1: 0.6936
Precision:   0.7044
Recall:      0.6843


# Inference Examples

In [ ]:
import re

def text_to_chat(chat_text: str):
    pattern = re.compile(r'(system|assistant|user)\s*', re.IGNORECASE)

    segments = pattern.split(chat_text)[1:]

    conversation = []
    for role, content in zip(segments[0::2], segments[1::2]):
        conversation.append((role.strip().capitalize(), content.strip()))

    for role, content in conversation:
        print(f"{role}: {content}\n")


In [ ]:
for prediction in predictions:

    print("\n========== New Chat ============\n")
    text_to_chat(prediction)


========== New Chat ============

System: Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Assistant: Hello

Assistant: What do you enjoy?

User: Hello

User: I really like sci-fi movies like The Matrix (1999)

Assistant: Very good

User: or The Terminator (1984)

Assistant: I really like The Matrix (1999)


========== New Chat ============

System: Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

User: Hello

User: Can you reccoment a movie

Assistant: Yes, Avengers is good

User: I really like The Wolf of Wall Street  (2013) Or American Hustle (2013)

User: I really dont like superheroes

User: can you reccomend something else

User: something like Memento  (2000) or Black Mass  (2015)

User: Also The Shawshank Redemption (1994) is really good morgan freeman did the role grat!

Assistant: yeah those are great movies

Assistant: I love Napoleon dynamite

User: Ok i will see it

User: i think thats a great comedy for a lazy weekend :D

User: Ok thank you 

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np
import nltk

nltk.download("punkt")

# --- MÉTRICA DE NOVEDAD ---
def calculate_novelty(predictions, references):
    novelty_scores = []
    for pred, ref in zip(predictions, references):
        pred_tokens = set(word_tokenize(pred.lower()))
        ref_tokens = set(word_tokenize(ref.lower()))
        novel_tokens = pred_tokens - ref_tokens
        novelty = len(novel_tokens) / (len(pred_tokens) + 1e-5)
        novelty_scores.append(novelty)
    return np.mean(novelty_scores)

# --- MÉTRICA SELF-BLEU ---
def compute_self_bleu(predictions):
    scores = []
    smoothing = SmoothingFunction().method1
    for i, pred in enumerate(predictions):
        hyp = word_tokenize(pred)
        refs = [word_tokenize(p) for j, p in enumerate(predictions) if j != i]
        if refs:  # evitar listas vacías
            score = sentence_bleu(refs, hyp, smoothing_function=smoothing)
            scores.append(score)
    return np.mean(scores)

novelty_score = calculate_novelty(generated_predicitons, references_message)
self_bleu_score = compute_self_bleu(generated_predicitons)

print(f"Novelty Score: {novelty_score:.4f}")
print(f"Self-BLEU Score: {self_bleu_score:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Novelty Score: 0.7621
Self-BLEU Score: 0.5616


In [ ]:
prompts = [
    "Can you recommend a horror movie?",
    "I don't know what to watch today, any ideas?",
    "I like sci-fi movies with a romantic touch.",
    "I watched Inception yesterday and loved it. Anything similar?",
    "Hey, what's a good movie to watch right now?",
    "Recomiéndame una película de terror.",
    "No sé qué ver hoy, ¿alguna idea?",
    "Me gustan las películas de ciencia ficción con un toque romántico.",
    "Ayer vi Inception y me encantó. ¿Algo similar?",
    "Ey, ¿qué peli está buena ahora?"
]

for i, prompt in enumerate(prompts):
    print(f"\n--- Prompt #{i+1} ---\n")

    messages = [
        {"role": "user", "content": prompt}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraer la respuesta del modelo desde el tag <|assistant|>
    if "<|assistant|>" in decoded:
        response = decoded.split("<|assistant|>")[-1].strip()
    else:
        response = decoded.strip()

    print(f"Prompt: {prompt}")
    print(f"Respuesta: {response}")